# Git для Data Science: контроль версий данных с помощью Python и DVC

---

Публикация представляет собой сокращенный перевод Кристиана Иванчича [Data Version Control With Python and DVC](https://realpython.com/python-data-version-control/).

---

Машинное обучение и наука о данных сопряжены с рядом задач, отличающихся от традиционной разработки программного обеспечения. [Системы управления версиями](https://ru.wikipedia.org/wiki/%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_%D1%83%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%8F%D0%BC%D0%B8) помогают разработчикам контролировать изменения исходного кода, однако управление версиями датасетов, изменения в моделях и наборах данных, отлажено не столь хорошо. Если датасет изменился, воспроизвести эксперимент становится сложной задачей. Поэтому многие команды активно разрабатывают инструменты и фреймворки для решения этих проблем.

В этом руководстве мы узнаем как:
- использовать инструмент под названием DVC для решения некоторых из этих проблем;
- корректно отслеживать и редактировать наборы данных и модели;
- совместно использовать одну систему с товарищами по команде;
- делать воспроизводимые эксперименты по машинному обучению.


# Что такое система управления версиями данных

В стандартной разработке программного обеспечения разработчикам приходится взаимодействовать с несколькими версиями одного и того же кода. Чтобы предотвратить путаницу и дорогостоящие ошибки, разработчики используют системы управления версиями, такие как [Git](https://proglib.io/p/git-for-half-an-hour).

В системе управления версиями есть центральный репозиторий кода, представляющий текущее состояние проекта. Разработчик может сделать копию проекта, внести изменения и запросить, чтобы они были добавлены в следующей официальной версии. В результате запроса код проверяется и тестируется, пока не будет выпущен.

Такие циклы повторяются по многу раз в день в проектах, связанных с разработкой. Но в мире промышленного Data Science подобные соглашения и стандарты пока не закрепились. Наличие систем, позволяющих людям работать быстро и продолжать с того места, на котором остановились другие, повысило бы скорость и качество получаемых результатов. Это помогло бы прозрачно управлять данными, эффективно проводить эксперименты и сотрудничать с другими людьми и командами.

Один из инструментов, помогающих исследователям управлять данными и осуществлять воспроизводимые эксперименты – **DVC** (сокр. от Data Version Control).


# Что такое DVC

[DVC](https://dvc.org/) – это написанный на Python инструмент командной строки, который  работает совместно с Git, а также имитирует команды и рабочие процессы Git, перенося аналогичные подходы на работу с данными. Фактически команды `git` и `dvc` часто используются вместе, одна за другой. В то время как Git используется для хранения версий кода, DVC делает то же самое для файлов моделей и датасетов.

Git может хранить код как локально, так и на Bitbucket, [GitHub или GitLab](https://proglib.io/p/gitlab-ili-github-kak-vybrat-resurs-pod-opredelennyy-tip-repozitoriya-2020-01-06). Аналогичным образом, DVC позволяет загружать данные на удаленный репозиторий для хранения данных и моделей. Вы можете создать локальную копию удаленного репозитория, изменить файлы, а затем загрузить свои изменения, чтобы поделиться ими с командой.

Удаленный репозиторий может находиться на том же компьютере, на котором вы работаете, или в облаке. DVC поддерживает большинство главных облачных провайдеров, включая AWS, GCP и Azure. Но вы можете настроить удаленный репозиторий DVC на любом другом сервере.

В процессе работы в удаленном репозитории создается `.dvc`-файл – небольшой текстовый файл с описанием текущего состояния файлов данных. Благодаря небольшому размеру `.dvc`-файл может храниться вместе с программным кодом, например, на GitHub.

Далее мы на практике изучим наиболее важные возможности DVC. Перед тем как начать, нужно настроить среду для работы, а затем получить некоторые данные.


# Настраиваем рабочую среду

В этом руководстве мы рассмотрим, как использовать DVC, попрактиковавшись на примерах работы с данными изображений. Мы даже обучим модель машинного обучения, которая будет распознавать, что содержит изображение.

Для работы с примерами в системе должны быть установлены [Python 3](https://realpython.com/installing-python/) и [Git](https://git-scm.com/book/en/v2/Getting-Started-Installing-Git).

Поскольку DVC – это инструмент командной строки, необходимо уметь работать в командной строке операционной системы. Если вы пользователь Windows, ознакомьтесь с разделом [«Запуск DVC в Windows» (англ.)](https://dvc.org/doc/user-guide/running-dvc-on-windows).

Чтобы подготовить рабочую среду, нужно сделать следующие шаги:
1. Создать и активировать виртуальное окружение.
2. Установить DVC и необходимые библиотеки Python.
3. Создайть форк и клонировать GitHub-репозиторий с кодом.
4. Загрузить набор данных для использования в примерах.

Вы можете использовать любой менеджер пакетов и окружений. В этом руководстве мы используем `conda`, поскольку он отлично поддерживает инструменты для анализа данных и машинного обучения. Чтобы создать и активировать виртуальную среду, откройте интерфейс командной строки и введите следующую команду:

```bash
conda create --name dvc python=3.8.2 -y
```

Команда `create` создает новую виртуальную среду. Флаг `--name` дает имя среде – в приведенном примере – `dvc`. Аргумент `python` позволяет выбрать версию Python, которую мы хотим установить в среде. Флаг `-y` автоматически соглашается установить все необходимые пакеты, которые нужны Python, без необходимости отвечать на какие-либо запросы.

Как только все установлено, активируем среду:

```bash
conda activate dvc
```

Теперь у нас есть среда Python, независимая от установленных в операционной системе библиотек Python. В нее мы установим внешние библиотеки:

- `dvc` – главный герой этого руководства;
- `scikit-learn` – библиотека machine learning для обучения моделей;
- `scikit-image` – библиотека обработки изображений, которую мы будем использовать для подготовки данных;
- `pandas` – библиотека анализа данных, представляющая данные в виде таблиц;
- `numpy` – библиотека, добавляющая поддержку многомерных данных.

Их тоже можно установить с помощью `conda`:

```bash
conda config --add channels conda-forge
conda install dvc scikit-learn scikit-image pandas numpy
```

В качестве альтернативы можно использовать установщик pip:

```bash
python -m pip install dvc scikit-learn scikit-image pandas numpy
```

Теперь нужно сделать форк репозитория [Data Version Control Tutorial](https://github.com/realpython/data-version-control) в свою учетную запись на GitHub. На странице репозитория GitHub нажмите `Fork` в правом верхнем углу экрана и выберите во всплывающем окне вашу учетную запись. GitHub создаст копию репозитория в вашей учетной записи.

Клонируем форк на свой компьютер с помощью команды `git clone` и переходим в папку репозитория (не забудьте поменять `YourUsername` на имя вашего аккаунта на GitHub):

```bash
git clone https://github.com/YourUsername/data-version-control
cd data-version-control
```

Cтруктура каталогов репозитория выглядит следующим образом:

```
data-version-control/
|
├── data/
│   ├── prepared/
│   └── raw/
|
├── metrics/
├── model/
└── src/
    ├── evaluate.py
    ├── prepare.py
    └── train.py

```

В репозитории шесть каталогов:

1. `src/` – для исходного кода;
2. `data/` – для всех версий набора данных;
3. `data/raw/` – для данных, полученных из внешнего источника;
4. `data/prepare/` – для данных, измененных внутри;
5. `model/` – для моделей машинного обучения;
6. `data/metrics/` – для отслеживания показателей производительности моделей.

Каталог `src/` уже содержит три файла Python:
1. `prepare.py` – код подготовки данных для обучения.
2. `train.py` – код обучения модели машинного обучения.
3. `evalueate.py` – код оценки результатов модели машинного обучения.

# Набор данных для обучения

Последний шаг в подготовке – получить пример набора данных, который мы можем использовать для практики DVC. Пример с изображениями подходит для этого лучше всего: управление большим количеством крупных файлов – это то, чем отличается DVC. Мы будем использовать набор данных [Imagenette](https://github.com/fastai/imagenette), предоставляемый компанией [fastai](https://www.fast.ai/).

Imagenette – это подмножество набора данных [ImageNet](http://www.image-net.org/), который часто используется в качестве эталонного набора данных во многих статьях по машинному обучению. ImageNet слишком велик, чтобы использовать его в качестве примера, поэтому мы возьмем его подмножество Imagenette. Перейдите на [GitHub-страницу Imagenette](https://github.com/fastai/imagenette) и нажмите ссылку для загрузки `160 px download`.

Это загрузит набор данных, сжатый в архив TAR. Размер файла – порядка 100 Мб. Пользователи Mac могут извлечь файлы, дважды щелкнув архив в Finder. Линуксоиды могут распаковать его с помощью команды `tar`. Пользователям Windows потребуется установить инструмент для распаковки файлов TAR, например [7-zip](https://www.7-zip.org/).

Набор данных структурирован определенным образом. В нем есть две основные папки:

1. `train/` – изображения для обучения модели.
2. `val/` – изображения для валидации модели.

---
**Примечание**. Валидация обычно происходит *во время обучения* модели – так аналитики понимают, насколько хорошо модель обучается. Поскольку это руководство не ориентировано на показатели производительности, мы будем использовать набор для проверки модели *после ее обучения*.

---

Imagenette – это набор обучающих данных для задачи классификации: каждое изображение имеет связанный с ним класс, описывающий, что на нем изображено. Чтобы решить задачу классификации, необходимо обучить модель, которая сможет точно определять класс изображения.

Каталоги `train/` и `val/` содержат несколько папок. Каждая из них соответствует одному из 10 классов:

1. Линь (вид лучеперых рыб)
2. Английский спрингер-спаниель (порода собак)
3. Кассетный проигрыватель
4. Цепная пила
5. Церковь
6. Валторна
7. Мусоровоз
8. Топливораздаточная колонка
9. Мячик для гольфа
10. Парашют

Для простоты и скорости в руководстве мы будем обучать модель, используя лишь два из десяти классов: мячик для гольфа и парашют. После обучения модель должна сообшать является ли то или иное изображение мячом для гольфа или фотографией парашюта. Такую задачу, когда модель выбирает между двумя типами объектов, называют [бинарной (или двоичной )классификацией](https://ru.wikipedia.org/wiki/%D0%94%D0%B2%D0%BE%D0%B8%D1%87%D0%BD%D0%B0%D1%8F_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F).

Переместим папки `train/` и `val/` в репозиторий data-version-control в каталог `data/raw/`. Структура репозитория теперь будет выглядеть так:

```bash
data-version-control/
|
├── data/
│   ├── prepared/
│   └── raw/
│       ├── train/
│       │   ├── n01440764/
│       │   ├── n02102040/
│       │   ├── n02979186/
│       │   ├── n03000684/
│       │   ├── n03028079/
│       │   ├── n03394916/
│       │   ├── n03417042/
│       │   ├── n03425413/
│       │   ├── n03445777/
│       │   └── n03888257/
|       |
│       └── val/
│           ├── n01440764/
│           ├── n02102040/
│           ├── n02979186/
│           ├── n03000684/
│           ├── n03028079/
│           ├── n03394916/
│           ├── n03417042/
│           ├── n03425413/
│           ├── n03445777/
│           └── n03888257/
|
├── metrics/
├── model/
└── src/
    ├── evaluate.py
    ├── prepare.py
    └── train.py
```

Отлично! Мы завершили настройку и готовы экспериментировать с DVC.

# Базовый рабочий процесс DVC

В этом разделе мы увидим, как DVC в тандеме с Git позволяет управлять и кодом, и данными.

Для начала создадим и переключимся на ветку нашего первого эксперимента:

```bash
git checkout -b "first_experiment"
```

`git checkout` изменяет текущую ветку, а переключатель `-b` сообщает Git, что этой ветки не существует и ее следует создать.

Далее нужно инициализировать DVC. Предварительно нужно убедиться, что мы находимся в папке верхнего уровня репозитория. Далее запускаем команду `dvc init`:

```bash
dvc init
```

Эта команда приведет к созданию папки `.dvc`, в которой хранится информация о конфигурации – аналогично тому, как каталог `.git` содержит данные Git. В принципе, вам никогда не нужно открывать эту папку, если только вы не хотите узнать, как работает dvc под капотом.

---

**Примечание**: При запуске DVC выдаст предупреждение, что собирает анонимную статистику использования (это помогает авторам улучшить инструмент). Вы можете отключить эту функцию, установив для параметра конфигурации аналитики значение `false`:

```bash
dvc config core.analytics false
```

---

Git позволяет отправить локальный код в удаленный репозиторий так, чтобы у всех участников проекта был единый источник информации. Коллеги могут проверить ваш код и поработать над ним локально, не опасаясь испортить текст программы для всех остальных. То же справедливо и для DVC.

Теперь нам нужно создать удаленное хранилище файлов данных и моделей, контролируемых DVC. Пока что для обучения это может быть просто другая папка в вашей системе. Создадим каталог за пределами  репозитория `data-version-control/` и назовем его `dvc_remote`.

Вернемся в репозиторий `data-version-control/` и укажем DVC, где находится удаленное хранилище:

```bash
dvc remote add -d remote_storage путь_к_dvc_remote
```

Теперь DVC знает, где хранить резервную копию данных и моделей. Команда `dvc remote add` сохраняет местоположение удаленного хранилища и называет его `remote_storage`. Если хотите, вы можете выбрать  для переменной другое имя. Ключ `-d` сообщает DVC, что это удаленное хранилище по умолчанию (`default`). Можно добавить и другие удаленные хранилища и переключаться между ними.

Внутри папки `.dvc` находится файл `config`, в котором хранится информация о конфигурации репозитория:

```
[core]
    analytics = false
    remote = remote_storage
['remote "remote_storage"']
    url = /path/to/your/remote_storage
```

DVC поддерживает облачные системы хранения: AWS S3, Google Cloud Storage, Microsoft Azure Blob Storage. Подключение описано в [официальной документации DVC](https://dvc.org/doc/command-reference/remote/add).

Основное практическое правило, которому мы должны следовать, заключается в том, что **небольшие файлы отправляются на GitHub, а большие – в удаленное хранилище DVC**.

# Отслеживание файлов

Для запуска отслеживания файлов и Git, и DVC используют команду `add`. Добавим папки `train/` и `val/` под управление DVC:

```
dvc add data/raw/train
dvc add data/raw/val
```

Вот что при этом делает DVC:

1. Добавляет каталоги `train/` и `val/` в `.gitignore`.
2. Создает два dvc-файла: `train.dvc` и `val.dvc`.
3. Копирует папки `train/` и `val/` в промежуточную область (staging).

`.gitignore` – это текстовый файл со списком файлов, которые Git не должен отслеживать. Когда файл указан в `.gitignore`, он невидим для команд `git`. Добавляя папки `train/` и `val/` в `.gitignore`, DVC гарантирует, что мы случайно не загрузим большие файлы данных на GitHub.

На следующем изображении показано, как выглядел репозиторий до выполнения каких-либо команд.

<img src="https://files.realpython.com/media/0_new-start.48d65eac9b3f.png" width=300>

Всё, что контролирует DVC, находится слева (зеленый цвет), а все, что контролирует Git, – справа (синий). Упрощенно будем отображать в репозитории файл с кодом `code.py` и папку `train/`.

Когда мы запускаем `dvc add train/`, папка с большими файлами переходит под контроль DVC, а маленькие dvc-файлы и файл `.gitignore` переходят под контроль Git. Папка `train/` попадает в staging-область DVC.

<img src="https://files.realpython.com/media/1_new-dvc_add.e7d290c59325.png" width=300>

После того как большие файлы изображений будут помещены под управление DVC, можно добавить весь код и небольшие файлы под управления Git с помощью `git add`:


```bash
git add --all
```

Теперь все файлы находятся под управлением соответствующих систем контроля версий.

<img src="https://files.realpython.com/media/2_new-git_add.2de3d14865f7.png" width=300>

Если кто-то хочет работать над проектом и использовать данные `train/` и `val/`, им сначала нужно загрузить репозиторий Git, затем использовать файлы `.dvc` для получения текущей версии данных.

Но прежде чем люди смогут получить репозиторий и данные, необходимо загрузить файлы в удаленное хранилище.

# Загрузка файлов

Чтобы загрузить файлы на GitHub, нужно сначала создать «снимок» текущего состояния репозитория:

```bash
git commit -m "Начало работы с DVC: настройка и загрузка DVC-файлов"
```

Ключ `-m` означает, что последующий текст в кавычках представляет собой сообщение, объясняющее, что было сделано.

В DVC также есть команда `commit`, но она делает не то же самое, что `git commit`. DVC не нужен снимок всего репозитория. Инструмент позволяет загружать файлы, как только они начинают отслеживаться с помощью `dvc add`. Команда `dvc commit` используется при изменении уже отслеживаемого файла. Внося локальное изменение в данные, мы должны зафиксировать изменение в кэше перед загрузкой на удаленный компьютер. Так как мы пока не меняли данные с момента их добавления, `commit` для dvc делать не нужно.

---
Подробнее об отличиях `add` и `commit` в DVC и Git рассказывает [документация DVC](https://dvc.org/doc/command-reference/add).

---

Чтобы загрузить файлы из кэша в удаленное хранилище, используем команду `push`:

```bash
dvc push
```

DVC просмотрит все папки локального репозитория в поисках dvc-файлов. Как уже упоминалось, эти файлы сообщают DVC, какие данные необходимо скопировать, и DVC копирует их из кэша в удаленное хранилище.


<img src="https://files.realpython.com/media/new-dvc_push_1.1e26af053dd1.png" width=300>

Теперь данные безопасно хранятся в удаленном репозитории. Осталось отправить файлы под управлением Git на GitHub:

```bash
git push --set-upstream origin first_experiment
```

GitHub не знает о новой ветке, которую мы создали локально, поэтому при первом `push` необходимо использовать параметр `--set-upstream`.


<img src="https://files.realpython.com/media/new-git_push_1.5ee03041c300.png" width=300>

Теперь и код, и данные доступны удаленно. 

# Скачивание файлов

Чтобы понять, как загружать файлы, удалим некоторые файлы из нашего репозитория.

Как только мы добавили данные с помощью `dvc add` и отправили их с помощью `dvc push`, они сохряются в резервной копии. Для экономии места вы можете удалить фактические данные. Пока все файлы отслеживаются DVC и dvc-файлы находятся в репозитории, мы можем быстро вернуть данные.

Например, мы можем удалить целиком папку `val/`:

```bash
rm -rf data/raw/val
```

Команда приведет к удалению папки `data/raw/val/` из репозитория, но каталог по-прежнему безопасно хранится в кэше и удаленном хранилище. Чтобы вернуть данные из кеша, воспользуемся командой `dvc checkout`:

```bash
dvc checkout data/raw/val.dvc
```

Папка `data/raw/val/` будет восстановлена. Если вы хотите, чтобы DVC выполнил поиск по всему репозиторию и проверил, чего не хватает, достаточно запустить `dvc checkout` без дополнительных аргументов.

Когда вы клонируете репозиторий GitHub на новый компьютер, кеш будет пустым. Команда `fetch` загружает содержимое удаленного хранилища в кеш:

```bash
dvc fetch data/raw/val.dvc
```

Также можно использовать `dvc fetch`, чтобы получить данные для всех файлов DVC в репозитории. Как только данные окажутся в кеше, проверьте их в репозитории с помощью `dvc checkout`. Еще можно выполнить вместе `fetch` и `checkout` с помощью одной команды `dvc pull`. Эта команда копирует данные с удаленного хранилища в кеш и в репозиторий за один проход. Действия команд аналогичны одноименным командам Git.

<img src="https://files.realpython.com/media/5_new-download.dd38540ba18f.png" width=400>

Имейте в виду, что сначала нужно получить dvc-файлы из Git, и только потом вызывать команды DVC. Если dvc-файлов нет в репозитории, программа DVC просто не будет знать, какие данные требуется получить.

Итак, мы рассмотрели базовый рабочий процесс взаимодействия DVC и Git. Всякий раз, когда мы добавляем данные или изменяем код, необходимо запустить соответствующие команды `add`, `commit`, `push`, чтобы сохранить текущую версию. Многим людям этого базового рабочего процесса будет достаточно для повседневных нужд.

Остальная часть этого руководства посвящена конкретным примерам использования DVC для машинного обучения и Data Science.

# Построение модели машинного обучения

Используя набор данных Imagenette, мы научим модель различать изображения мячей для гольфа и парашютов. Для этого мы выполним три шага:

1. Подготовим данные для обучения.
2. Обучим модель машинного обучения.
3. Оценим качество модели.

Эти шаги соответствуют трем файлам Python в папке `src/`:

1. `prepare.py`
2. `train.py`
3. `evaluate.py`

В следующих подразделах мы рассмотрим, что делает каждый из файлов.

## Подготовка данных

Данные хранятся в нескольких каталогах. Чтобы упростить использование данных, мы создадим csv-файл, содержащий список изображений и их меток. CSV-файл будет содержать два столбца: столбец `filename`, содержащий полный путь файла конкретного изображения, и столбец `label`, содержащий строку метки, например `"golf ball"` или `"parachute"`. Каждая строка в csv файле соответствует одному изображению.

```csv
filename, label
full/path/to/data-version-control/raw/n03445777/n03445777_5768.JPEG,golf ball
full/path/to/data-version-control/raw/n03445777/n03445777_5768,golf ball
full/path/to/data-version-control/raw/n03445777/n03445777_11967.JPEG,golf ball
...
```

Далее понадобится два CSV-файла:

- `train.csv` со списком изображений для обучения.
- `test.csv` со списком изображений для тестирования.

Мы можем создать CSV-файлы, запустив программу `prepare.py`, которая сопоставляет имена папок и метки и сохраняет пары список-метка в виде CSV-файла.

Исходный код, используемый на этапе подготовки, следующий:

```python
# prepare.py
from pathlib import Path

import pandas as pd

# имена папок, содержащие изображения мячей для гольфа и парашютов,
# сопоставлены с метками "golf ball" и "parachute"
FOLDERS_TO_LABELS = {
    "n03445777": "golf ball",
    "n03888257": "parachute"
    }

def get_files_and_labels(source_path):
    """Принимает путь, указывающий на папку `data/raw/`.
    Функция перебирает все папки и подпапки, чтобы найти файлы
    с расширением jpeg. Метки присваиваются тем файлам, папки
    которых представлены в виде ключей в FOLDERS_TO_LABELS.
    Имена файлов и метки возвращаются в виде списков."""
    images = []
    labels = []
    for image_path in source_path.rglob("*/*.JPEG"):
        filename = image_path.absolute()
        folder = image_path.parent.name
        if folder in FOLDERS_TO_LABELS:
            images.append(filename)
            label = FOLDERS_TO_LABELS[folder]
            labels.append(label)
    return images, labels

def save_as_csv(filenames, labels, destination):
    """Принимает список файлов, список меток и путь назначения.
    Имена файлов и метки форматируются как датафрейм pandas
    и сохраняются в виде csv-файла."""
    data_dictionary = {"filename": filenames, "label": labels}
    data_frame = pd.DataFrame(data_dictionary)
    data_frame.to_csv(destination)

def main(repo_path):
    """Запускает get_files_and_labels(), чтобы найти
    все изображения в папках data/raw/train/ и data/raw/val/.
    Имена файлов и соответствующие им метки сохраняются
    как два csv-файла в папке data/prepare/: train.csv
    и test.csv."""
    data_path = repo_path / "data"
    train_path = data_path / "raw/train"
    test_path = data_path / "raw/val"
    train_files, train_labels = get_files_and_labels(train_path)
    test_files, test_labels = get_files_and_labels(test_path)
    prepared = data_path / "prepared"
    save_as_csv(train_files, train_labels, prepared / "train.csv")
    save_as_csv(test_files, test_labels, prepared / "test.csv")

if __name__ == "__main__":
    repo_path = Path(__file__).parent.parent
    main(repo_path)
```



Запустите скрипт `prepare.py` в командной строке:

```bash
python src/prepare.py
```

Когда скрипт закончит работу, в папке `data/prepare/` появятся файлы `train.csv` и `test.csv`. Нужно будет добавить их в DVC, а соответствующие .dvc`-файлы в GitHub:

```bash
dvc add data/prepared/train.csv data/prepared/test.csv
git add --all
git commit -m "Created train and test CSV files"
```

Теперь у нас есть список файлов, которые можно использовать для обучения и тестирования модели машинного обучения.

# Обучение модели

Для обучения модели мы будем использовать один из простейших алгоритмов обучения с учителем – стохастический градиентный спуск. Исходный код, который мы будем использовать на этапе обучения:

```python
# train.py
from joblib import dump
from pathlib import Path

import numpy as np
import pandas as pd
from skimage.io import imread_collection
from skimage.transform import resize
from sklearn.linear_model import SGDClassifier

def load_images(data_frame, column_name):
    filelist = data_frame[column_name].to_list()
    image_list = imread_collection(filelist)
    return image_list

def load_labels(data_frame, column_name):
    label_list = data_frame[column_name].to_list()
    return label_list

def preprocess(image):
    resized = resize(image, (100, 100, 3))
    reshaped = resized.reshape((1, 30000))
    return reshape

def load_data(data_path):
    df = pd.read_csv(data_path)
    labels = load_labels(data_frame=df, column_name="label")
    raw_images = load_images(data_frame=df, column_name="filename")
    processed_images = [preprocess(image) for image in raw_images]
    data = np.concatenate(processed_images, axis=0)
    return data, labels

def main(repo_path):
    train_csv_path = repo_path / "data/prepared/train.csv"
    train_data, labels = load_data(train_csv_path)
    sgd = SGDClassifier(max_iter=10)
    trained_model = sgd.fit(train_data, labels)
    dump(trained_model, repo_path / "model/model.joblib")

if __name__ == "__main__":
    repo_path = Path(__file__).parent.parent
    main(repo_path)
```

Теперь запустим скрипт `train.py` в командной строке:

```bash
python src/train.py
```

Выполнение кода может занять несколько минут, в зависимости от мощности вашего компьютера. Вы можете получить предупреждение при выполнении этого кода:

`ConvergenceWarning: Maximum number of iteration reached before convergence.
Consider increasing max_iter to improve the fit.`


Это означает, что `scikit-learn` считает, что вы можете увеличить `max_iter` и получить лучшие результаты. Вы сделаете это в одном из следующих разделов, но цель этого руководства состоит в том, чтобы эксперименты выполнялись быстро, а не имели максимально возможную точность.

Когда скрипт завершится, у нас будет обученная модель машинного обучения, сохраненная в папке `model/` с именем `model.joblib`. Это самый важный файл эксперимента. Его необходимо добавить в DVC с привязкой соответствующего файла `.dvc` к GitHub:

```bash
dvc add model/model.joblib
git add --all
git commit -m "Trained an SGD classifier"
```

Мы обучили модель машинного обучения различать два класса изображений. Следующий шаг – определить, насколько точно модель работает на тестовых изображениях, которые модель не видела во время обучения.

# Оценка модели

Проверка работы модели служит своего рода наградой – вы, наконец, видите результат усилий. Вот исходный код, который мы используем на этапе оценки:

```python
# evaluate.py
from joblib import load
import json
from pathlib import Path

from sklearn.metrics import accuracy_score

from train import load_data

def main(repo_path):
    test_csv_path = repo_path / "data/prepared/test.csv"
    test_data, labels = load_data(test_csv_path)
    model = load(repo_path / "model/model.joblib")
    predictions = model.predict(test_data)
    accuracy = accuracy_score(labels, predictions)
    metrics = {"accuracy": accuracy}
    accuracy_path = repo_path / "metrics/accuracy.json"
    accuracy_path.write_text(json.dumps(metrics))

if __name__ == "__main__":
    repo_path = Path(__file__).parent.parent
    main(repo_path)
```

В командной строке запустиим `evaluate.py`:

```bash
python src/evaluate.py
```

После запуска модели ее оценка будет сохранена в файле `metics/accuracy.json`. Каждый раз, когда вы меняете что-то в модели или используете другую, вы можете увидеть, улучшается ли она, сравнив ее с этим значением.

Файл JSON сейчас содержит только один объект, точность (`accuracy`) модели, например:

```JSON
{"accuracy": 0.7186311787072244}
```

Файл JSON очень небольшой, и его полезно хранить на GitHub, чтобы можно было быстро проверить, насколько хорошо выполняется каждый эксперимент:

```bash
git add --all
git commit -m "Evaluate the SGD model accuracy"
```

После завершения оценки мы готовы углубиться в некоторые расширенные функции и процессы DVC.


# Версии датасетов и моделей
В основе воспроизводимой науки о данных лежат «снимки» всего, что используется для построения модели. Каждый раз, когда мы запускаем эксперимент, мы хотим точно знать, что было на входе, и что получилось на выходе.

В этом разделе мы познакомимся с более сложным рабочим процессом для контроля версий в наших экспериментах. 

Сначала отправим все изменения, внесенные нами в ветку `first_experiment`, в удаленные хранилища GitHub и DVC:

```bash
git push
dvc push
```

Обучение модели и завершение эксперимента – важная веха для проекта. Поэтому мы должны следить за возможностью оперативно возвращаться к таким точкам.


# Добавление тегов к коммитам

Распространенной практикой является использование тегов, чтобы отметить определенный момент в истории Git. Поскольку мы завершили эксперимент и создали новую модель, создадим тег, чтобы сообщить себе и другим, что у нас есть готовая модель:

```bash
git tag -a sgd-classifier -m "SGDClassifier with accuracy 71.86%"
```

Ключ `-a` используется для аннотирования тега. Можно использовать любое удобное описание. Некоторые команды вводят номера версий, например `v1.0`, `v1.3`. Другие используют даты и инициалы члена команды, обучавшего модель. Ключ `-m` позволяет добавить в тег строку сообщения, как и в случае с коммитами.

Теги Git не передаются, как обычные коммиты, поэтому их нужно отправлять отдельно в источник репозитория на GitHub. Используйте переключатель `--tags`, чтобы передать все теги из локального репозитория на удаленный:

```bash
git push origin --tags
```

Если вы используете GitHub, теги доступны на вкладке `Releases` вашего репозитория.

Теги текущего репозитория выводятся так:

```bash
git tag
```

Рабочие процессы DVC в значительной степени зависят от эффективных практик Git.  Еще один способ сделать рабочий процесс более упорядоченным и прозрачным – использовать ветвление.


# Создание отдельных веток Git для каждого эксперимента

Пока что мы сделали всю работу в ветке `first_experiment`. Сложные задачи и долгосрочные проекты обычно требуют проведения множества экспериментов. Хорошая практика – создавать отдельную ветку для каждого эксперимента.

В нашем первом эксперименте мы установили максимальное количество итераций модели равным `10`. Мы можем попробовать увеличить это число, чтобы увидеть, улучшит ли это результат.

```bash
git checkout -b "sgd-100-iterations"
```

Когда мы создаем новую ветку, все файлы `.dvc`, которые были в предыдущей ветке, будут присутствовать в новой ветке, как и другие файлы и папки.

Обновите код в `train.py` так, чтобы модель `SGDClassifier` запускалась с параметром `max_iter = 100`.

```python
# train.py
def main(repo_path):
    train_csv_path = repo_path / "data/prepared/train.csv"
    train_data, labels = load_data(train_csv_path)
    sgd = SGDClassifier(max_iter=100) # <-- обновим значение
    trained_model = sgd.fit(train_data, labels)
    dump(trained_model, repo_path / "model/model.joblib")
```

Это единственное изменение, которое необходимо сделать. Далее повторите обучение и оценку, запустив `train.py` и `evaluate.py`:

```bash
python src/train.py
python src/evaluate.py
```

При этом обновятся файлы `model.joblib` и `precision.json`. Так что необходимо зафиксировать результаты в кэше DVC:

```bash
dvc commit
```

DVC выведет вопрос, действительно ли мы хотим внести изменения, отвечаем `y`.

Напомним, что `dvc commit` работает иначе, чем `git commit`, и используется для обновления уже отслеживаемого файла. Это не удалит предыдущую модель, а создаст новую.

Добавим и зафиксируем внесенные изменения в Git:

```bash
git add --all
git commit -m "Change SGD max_iter to 100"
```

Добавим тег:

```bash
git tag -a sgd-100-iter -m "Trained an SGD Classifier for 100 iterations"
git push origin --tags

```

Отправим изменения на GitHub и в удаленное хранилище DVC:

```bash
git push --set-upstream origin sgd-100-iter
dvc push
```

Теперь можно переключаться между ветками Git и DVC.


```bash
git checkout first_experiment
dvc checkout
```

Отлично! Теперь у нас есть несколько экспериментов, а их результаты представлены в виде разных версий, и мы можем оперативно получать доступ и к коду, и к разным версиям моделей.


# Что скрывается внутри DVC-файлов
Мы создали и зафиксировали несколько файлов `.dvc` на GitHub, но что внутри них? Откройем текущий `dvc`-файл  модели: `data-version-control/model/model.joblib.dvc`. Его содержание примерно следующее:

```yaml
md5: 62bdac455a6574ed68a1744da1505745
outs:
  - md5: 96652bd680f9b8bd7c223488ac97f151
    path: model.joblib
    cache: true
    metric: false
    persist: false
```

Содержание может сбить с толку: файлы DVC – это фактически файлы c YAML-разметкой. Информация хранится в парах ключ-значение и списках. Первый ключ – `md5`, за которым следует строка, казалось бы, случайных символов.

[MD5](https://ru.wikipedia.org/wiki/MD5) – популярный алгоритм хеширования. [Хеширование](https://ru.wikipedia.org/wiki/%D0%A5%D0%B5%D1%88-%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F) использует содержимое файла произвольного размера, чтобы создать строку символов фиксированной длины. Такая строка называется хешем или [контрольной суммой](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D1%82%D1%80%D0%BE%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%81%D1%83%D0%BC%D0%BC%D0%B0). Длина строки – независимо от размера исходного файла – всегда составляет 32 символа.

Два одинаковых файла дают одинаковый хеш. Если в одном из файлов изменится хотя бы один бит, хеши перестанут совпадать. DVC использует эти свойства MD5 для достижения двух важных целей:
- чтобы отслеживать, какие файлы были изменены, просто взглянув на их хеш-значения
- чтобы узнать, когда два больших файла совпадают, чтобы в кеше или удаленном хранилище можно было сохранить только одну копию

В рассматриваемом примере есть два значения md5. Первый описывает сам файл `.dvc`, а второй – файл `model.joblib.path` – путь к файлу модели относительно рабочего каталога. Логическое значение `cache` определяет, должен ли DVC кэшировать модель.

Дополнительные сведения о dvc-файлах можно почерпнуть в [официальной документации](https://dvc.org/doc/user-guide/dvc-file-format).


# Совместная работа

Только что освоенного рабочего процесса достаточно, если вы единственный, кто использует оборудование, на котором провоядстя эксперименты. Однако многим командам для работы приходится совместно использовать мощные машины.

Когда с данными работают несколько пользователей, не хочется плодить  множество копий одних и тех же датасетов. Для экономии места DVC позволяет настроить общий кеш. Когда мы инициализируем репозиторий DVC с помощью `dvc init`, DVC помещает кеш в папку `.dvc/cache`. Этот путь можно изменить – для наглядности создадим новую папку `shared_cache` где-нибудь за пределами папки репозитория. Укажем DVC использовать эту папку в качестве кеша:

```bash
dvc cache dir путь_к_shared_cache
```

Теперь каждый раз, когда мы запускаем `dvc add` или `dvc commit`, данные будут копироваться в эту папку. Когда мы используем `dvc fetch` для получения данных из удаленного хранилища, они попадают в общий кеш, а `dvc checkout` переносит их в ваш рабочий репозиторий.

Если вы следовали и работали с примерами этого руководства, то все файлы сейчас находятся в папке `.dvc/cache` вашего репозитория. После выполнения указанной команды логично переместить данные из кеша по умолчанию в новый общий кеш:

```bash
mv .dvc/cache/* путь_к_shared_cache
```

Теперь все пользователи на этом компьютере могут указать свои кеши репозитория как общий кеш:

<img src="https://files.realpython.com/media/6_new-shared_cache.54170a671825.png" width="300">

Если вы проверите файл своего репозитория `.dvc/config`, то увидите, что появится новый раздел:


```bash
[cache]
    dir = путь_к_shared_cache
```

Но как это помогает сэкономить место? Вместо того, чтобы иметь копии одних и тех же данных в локальном репозитории, общем кэше и других репозиториях на машине, DVC позволяет использовать **ссылки** – типа `reflink`, `symlink` (символические) или `hardlink` (жесткие ссылки). DVC будет пытаться использовать по умолчанию `reflink`, однко если ОС не поддерживает рефссылки, DVC будет создавать копии. Больше о типах файловых ссылок можно узнать в [документации DVC](https://dvc.org/doc/user-guide/large-dataset-optimization#file-link-types-for-the-dvc-cache).

Если кратко, то поведение кеша по умолчанию можно изменить, указав параметр конфигурации `cache.type`, подставив вместо `тип_ссылки` значение `symlink`, `reflink`, `hardlink` или `copies`:


```bash
dvc config cache.type тип_ссылки
dvc checkout --relink
```

Переключатель `--relink` указывает DVC проверить тип кеша и повторно связать все файлы,  отслеживаемые DVC.

Если в вашем репозитории или кеше есть модели или файлы данных, которые не используются, вы можете сэкономить дополнительное пространство, очистив репозиторий с помощью сборщика мусора [dvc gc](https://dvc.org/doc/command-reference/gc).


# Создаем воспроизводимый конвейер Data Science

Вот краткий обзор шагов, которые мы сделали к настоящему времени для обучения модели машинного обучения:

1. Получение данных
2. Подготовка данных
3. Обучение модели
4. Оценка результатов обучения

Вы могли заметить, что при изменении параметров некоторые шаги мы повторяли вручную. Этот процесс можно сделать удобнее, объединив последовательность действий в **конвейер DVC**, запускаемый единственной командой.

Создайте новую ветку и назовем ее `sgd-pipeline`:


```bash
git checkout -b sgd-pipeline
```

Используем эту ветку, чтобы повторно запустить эксперимент в виде конвейера DVC. Конвейер состоит из нескольких этапов и выполняется с помощью команды `dvc run`. Каждый этап состоит из трех компонентов:

1. Входные объекты
2. Выходные объекты
3. Выполняемая команда

DVC использует термины для входов `dependencies` и `outs` для выходов. Командой (`command`) может быть все, что мы обычно запускаем в командной строке, в том числе файлы Python.

---

**Примечание**. В ходе выполнения конвейера будут заново созданы файлы, созданные с помощью `prepare.py`, `train.py` и `evaluate.py`. Конвейер автоматически добавляет вновь созданные файлы в элемент управления DVC, как если бы вы набрали `dvc add`. Поскольку мы успели уже вручную добавить много файлов в управление DVC, DVC запутается, если мы попытаемся создать те же файлы с помощью конвейера. Чтобы этого избежать, сначала удалим CSV-файлы, модели и показатели с помощью `dvc remove`:


```bash
dvc remove data/prepared/train.csv.dvc \
             data/prepared/test.csv.dvc \
             model/model.joblib.dvc --outs
```
---

Итак, мы начинаем конвейер с запуска `prepare.py`. Передаем команде `dvc run` необходимые данные:

1. **Dependencies** (ключ `-d`): `prepare.py` и данные в `data/raw`
2. **Outs** (ключ `-o`): `train.csv` и `test.csv`
3. **Command**: `python prepare.py`

Описываем, ключ `-n` используем для создания имени этапа:


```bash
dvc run -n prepare \
        -d src/prepare.py -d data/raw \
        -o data/prepared/train.csv -o data/prepared/test.csv \
        python src/prepare.py
```

DVC создаст два файла: `dvc.yaml` и `dvc.lock`. Пример того, что можно увидеть в `dvc.yaml`:

```yaml
stages:
  prepare:
    cmd: python src/prepare.py
    deps:
      - data/raw
      - src/prepare.py
    outs:
      - data/prepared/test.csv
      - data/prepared/train.csv
```

Элемент верхнего уровня, `stages`, имеет вложенные элементы, по одному для каждого этапа. На данный момент у нас только один этап `prepare`. По мере того, как мы будем наращивать конвейер, в файле будут добавляться элементы. Технически не обязательно вводить команды `dvc run` в командной строке, а достаточно создавать или варьировать этапы здесь.

У каждого `dvc.yaml` есть соответствующий файл `dvc.lock`, также в формате `YAML`. Информация внутри аналогична, с добавлением хэшей MD5 для всех зависимостей и выходов:

```yaml
prepare:
  cmd: python src/prepare.py
  deps:
    - path: data/raw
      md5: a8a5252d9b14ab2c1be283822a86981a.dir
    - path: src/prepare.py
      md5: 0e29f075d51efc6d280851d66f8943fe
  outs:
    - path: data/prepared/test.csv
      md5: d4a8cdf527c2c58d8cc4464c48f2b5c5
    - path: data/prepared/train.csv
      md5: 50cbdb38dbf0121a6314c4ad9ff786fe
```

Добавление хэшей MD5 позволяет DVC отслеживать все зависимости и выходные данные и определять, изменяется ли какой-либо из этих файлов. Таким образом, вместо отдельных файлов `.dvc` для `train.csv`, `test.csv` и `model.joblib`, все отслеживается в файле `.lock`.

Мы автоматизировали первый этап конвейера, который можно представить в виде блок-схемы:

<img drc="https://files.realpython.com/media/new-pipeline_1_1.79fcfdf683a8.png" width="300">

Следующий этап – обучение:
```bash
dvc run -n train \
        -d src/train.py -d data/prepared/train.csv \
        -o model/model.joblib \
        python src/train.py
```

<img drc="https://files.realpython.com/media/new-pipeline_2_1.dca61ebc0e0e.png" width="300">

Финальный этап – оценка модели:

```bash
dvc run -n evaluate \
        -d src/evaluate.py -d model/model.joblib \
        -M metrics/accuracy.json \
        python src/evaluate.py
```

Обратите внимание, что здесь вместо ключа `-o` мы использовали ключ `-M`. DVC обрабатывает метрики иначе, чем другие выходные данные. Когда мы запускаем эту команду, она генерирует файл `accuracy.json`, но DVC будет знать, что это показатель, используемый для измерения производительности модели. Это позволяет удобно обращаться к DVC для отображения метрики, например:

```bash
dvc metrics show
    metrics/accuracy.json:
        accuracy: 0.6996197718631179
```

<img drc="https://files.realpython.com/media/new-pipeline_3_1.2459d31b98e5.png" width="300">

Теперь весь рабочий процесс представлен на одном изображении. Не забудьте сделать тег для новой ветки и отправить все изменения на GitHub и DVC:


```bash
git commit -m "Rerun SGD as pipeline"
dvc commit
git push --set-upstream origin sgd-pipeline
git tag -a sgd-pipeline -m "Trained SGD as DVC pipeline."
git push origin --tags
dvc push
```

А теперь самое интересное! Воспользуемся для обучения классификатором **random forest**. Обычно он работает эффективнее, чем `SGDClassifier`, и потенциально может дать лучшие результаты. Начнем с создания и проверки новой ветки, которую назовем `random_forest`:


```bash
git checkout -b "random_forest"
```

Изменим `src/train.py`, чтобы использовать `RandomForestClassifier` вместо `SGDClassifier`:


```python
from joblib import dump
from pathlib import Path

import numpy as np
import pandas as pd
from skimage.io import imread_collection
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier  # <- 
# ...

def main(path_to_repo):
    train_csv_path = repo_path / "data/prepared/train.csv"
    train_data, labels = load_data(train_csv_path)
    rf = RandomForestClassifier()                  # <- 
    trained_model = rf.fit(train_data, labels)     # <- 
    dump(trained_model, repo_path / "model/model.joblib")
```

Поскольку файл `train.py` изменился, стал другим и его хеш MD5. DVC поймет, что необходимо воспроизвести одну из стадий конвейера. Поскольку изменение модели повлияет и на метрику, мы хотим воспроизвести всю цепочку. Любой этап конвейера DVC можно воспроизвести с помощью команды `dvc repro`:

```bash
dvc repro evaluate
```

**И всё!** Когда мы запускаем команду `repro`, DVC проверяет все зависимости всего конвейера, чтобы определить, что изменилось и какие команды нужно выполнить снова. Можно перемещаться между ветками и воспроизводить любой эксперимент с помощью одной лишь команды!

Кроме того, очень просто сравнивать метрики. Если запустить `dvc metrics show` с ключом `-T`, будут отображаться метрики для всех тегов.

```yaml
dvc metrics show -T
forest:
    metrics/accuracy.json:
        accuracy: 0.8098859315589354
sgd-pipeline:
    metrics/accuracy.json:
        accuracy: 0.6996197718631179

```

Это позволяет быстро определить, какой эксперимент в репозитории дал наилучший результат. Когда вы вернетесь к проекту спустя полгода и уже забудете о всех подробностях, вы можете проверить, какая настройка была наиболее успешной с помощью параметра `dvc metrics show -T` и тут же воспроизвести результат с помощью `dvc repro`. Также, как любой другой человек, который хочет воспроизвести вашу работу. Вашим коллегам достаточно выполнить три шага:

1. Запустить `git clone` или `git checkout`, чтобы получить программный код и файлы `.dvc`.
2. Получить данные обучения с помощью `dvc checkout`.
3. Воспроизведите рабочий процесс с помощью команды `dvc repro evaluate`.


# Заключение

Поздравляем с прохождением туториала! Итак, мы провели несколько экспериментов, обеспечили безопасное создание версий, резервное копирование данных и моделей. Более того, мы можем быстро воспроизвести каждый эксперимент, просто получив необходимый код и данные и выполнив одну команду `dvc repro`.

Поначалу может показаться несколько сложным запускать в нужные моменты все команды DVC и Git. Положение могут облегчить [хуки Git](https://dvc.org/doc/command-reference/install) – при запуске определенных команд Git автоматически выполнятся команды DVC.

DVC имеет [Python API](https://dvc.org/doc/api-reference) – вы можете вызывать команды DVC прямо из кода Python.

Несмотря на то, что это руководство представляет собой достаточно подробный обзор возможностей DVC, невозможно охватить все в одной публикации. Вы можете подробно изучить DVC, ознакомившись с [официальным руководством пользователя](https://dvc.org/doc/user-guide), [справочником команд](https://dvc.org/doc/command-reference) и [интерактивными учебниками](https://dvc.org/doc/tutorials/interactive).